In [3]:
import numpy as np

R=np.array([[4,np.NaN,np.NaN,2,np.NaN],
             [np.NaN,5,np.NaN,3,1],
             [np.NaN,np.NaN,3,4,4],
             [5,2,1,2,np.NaN]])
num_users,num_items=R.shape
K=3

np.random.seed(1)
P=np.random.normal(scale=1./K,size=(num_users,K))
Q=np.random.normal(scale=1./K,size=(num_items,K))             

In [4]:
from sklearn.metrics import mean_squared_error

def get_rmse(R,P,Q,non_zeros):
    error=0
    
    full_pred_matrix=np.dot(P,Q.T)
    
    x_non_zero_ind=[non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind=[non_zero[1] for non_zero in non_zeros]
    R_non_zeros=R[x_non_zero_ind,y_non_zero_ind]
    full_pred_matrix_non_zeros=full_pred_matrix[x_non_zero_ind,y_non_zero_ind]
    mse=mean_squared_error(R_non_zeros,full_pred_matrix_non_zeros)
    rmse=np.sqrt(mse)
    
    return rmse


In [8]:
non_zeros=[(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]
steps=1000
learning_rate=0.01
r_lambda=0.01

for step in range(steps):
    for i,j,r in non_zeros:
        eij=r-np.dot(P[i,:],Q[j,:].T)
        
        P[i,:]=P[i,:]+learning_rate*(eij*Q[j,:]-r_lambda*P[i,:])
        Q[j,:]=Q[j,:]+learning_rate*(eij*P[i,:]-r_lambda*Q[j,:])
        rmse=get_rmse(R,P,Q,non_zeros)
    if(step %50)==0:
        print('### iteration step :',step,"rmse :",rmse)
    

### iteration step : 0 rmse : 0.016326493769436592
### iteration step : 50 rmse : 0.01628946907391542
### iteration step : 100 rmse : 0.01625118644561044
### iteration step : 150 rmse : 0.016211966345828437
### iteration step : 200 rmse : 0.016172099816130634
### iteration step : 250 rmse : 0.01613183961187999
### iteration step : 300 rmse : 0.016091398951149288
### iteration step : 350 rmse : 0.016050954016130718
### iteration step : 400 rmse : 0.016010648074153127
### iteration step : 450 rmse : 0.015970596067220517
### iteration step : 500 rmse : 0.015930889075884927
### iteration step : 550 rmse : 0.01589159837695399
### iteration step : 600 rmse : 0.015852778988246913
### iteration step : 650 rmse : 0.015814472686837135
### iteration step : 700 rmse : 0.015776710534178725
### iteration step : 750 rmse : 0.015739514962320594
### iteration step : 800 rmse : 0.015702901481734373
### iteration step : 850 rmse : 0.015666880069856125
### iteration step : 900 rmse : 0.01563145629434261
#

In [9]:
pred_matrix=np.dot(P,Q.T)
print(pred_matrix)

[[3.9912571  1.14616211 1.27402561 2.00016417 1.6636353 ]
 [6.22825783 4.97759492 0.89267777 2.98269821 1.00309742]
 [6.39853978 0.8766627  2.98707787 3.97791848 3.98584323]
 [4.96929116 2.00492129 1.00638672 2.01249342 1.26216292]]
